# Result Arrangement

In [25]:
import pandas as pd
from glob import glob
import os
import pickle
import sys
import re

import numpy as np
import matplotlib.pyplot as plt

## Category 

In [10]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


## Path

In [12]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'
outpath = './outcome/'

In [15]:
outcomeFileList = glob(outpath+'outcome_news_classification'+'*')

In [66]:
for outcomeFile in outcomeFileList:
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = None, encoding='utf-8')
    modelIs = re.search('(Doc2Vec)|(word2vec)|(fastText)', fileName)
    modelIs = modelIs.group()
    if modelIs == 'Doc2Vec':
        modelType = re.search('(dbow)|(dm-c)|(dm-m)', fileName)
        modelType = modelType.group()
        
    elif modelIs == 'word2vec':
        modelType1 = re.search('(sg-[0-1])',fileName)
        modelType1 = modelType1.group()
        if re.search('[0-1]', modelType1).group() =='1':
            modelType1 = 'skip-gram'
        else:
            modelType1 = 'CBOW'
        modelType2 = re.search('cbow_mean-[0-1]', fileName)
        modelType2 = modelType2.group()
        modelType = modelType1 + '_'+modelType2
        del modelType1,modelType2
    elif modelIs == 'fastText':
        modelType1 = re.search('(sg-[0-1])', fileName)
        modelType1 = modelType1.group()
        if re.search('[0-1]', modelType1).group() =='1':
            modelType1 = 'skip-gram'
        else:
            modelType1 = 'CBOW'
        modelType2 = re.search('cbow_mean-[0-1]', fileName)
        modelType2 = modelType2.group()
        modelType = modelType1 + '_'+modelType2
        del modelType1,modelType2
    print (modelIs, modelType)

Doc2Vec dbow
fastText dbow
Doc2Vec dm-c
Doc2Vec dm-c
Doc2Vec dbow
Doc2Vec dm-m
word2vec CBOW
fastText CBOW
word2vec CBOW
fastText CBOW
word2vec CBOW
word2vec CBOW
fastText CBOW
Doc2Vec dm-m
word2vec skip-gram
fastText skip-gram
fastText skip-gram
word2vec skip-gram


<_sre.SRE_Match object; span=(3, 4), match='1'>

<_sre.SRE_Match object; span=(82, 86), match='sg-1'>

In [55]:
x.group()

'word2vec'